In [16]:
import pandas as pd
import openassetpricing as openap

# Define the signals you want to download
# signallist = ['IndIPO', 'BM']
signallist = ['BM', 'Mom12m']

# Initialize OpenAssetPricing API object
openap_obj = openap.OpenAP()

# Download portfolio returns for the selected signals
port_osap = openap_obj.dl_port('op', 'pandas', signallist)

# Convert date column to datetime
port_osap['date'] = pd.to_datetime(port_osap['date'])

# Keep only long-short (LS) portfolios
port_osap = port_osap[port_osap['port'] == 'LS']

# Display the result
port_osap



Data is downloaded: 5s


,signalname,port,date,ret,signallag,Nlong,Nshort
8700,BM,LS,1951-07-31,-1.115882,NaN,33,33
8701,BM,LS,1951-08-31,2.060248,NaN,33,33
8702,BM,LS,1951-09-28,2.272952,NaN,33,33
8703,BM,LS,1951-10-31,0.952421,NaN,33,33
8704,BM,LS,1951-11-30,0.388612,NaN,33,33
...,...,...,...,...,...,...,...
22369,Mom12m,LS,2023-08-31,9.140462,NaN,487,477
22370,Mom12m,LS,2023-09-29,7.178104,NaN,484,466
22371,Mom12m,LS,2023-10-31,6.032221,NaN,485,484
22372,Mom12m,LS,2023-11-30,6.939888,NaN,477,472


In [17]:
port_osap['signalname'].unique()


array(['BM', 'Mom12m'], dtype=object)

In [18]:
# Keep only relevant columns
port_osap_clean = port_osap[['signalname', 'date', 'ret']].copy()

# Sort and calculate cumulative returns grouped by signal
port_osap_clean = port_osap_clean.sort_values(['signalname', 'date'])
port_osap_clean['cum_return'] = port_osap_clean.groupby('signalname')['ret'].transform(lambda x: (1 + x / 100).cumprod())

# Display result
port_osap_clean

,signalname,date,ret,cum_return
8700,BM,1951-07-31,-1.115882,0.988841
8701,BM,1951-08-31,2.060248,1.009214
8702,BM,1951-09-28,2.272952,1.032153
8703,BM,1951-10-31,0.952421,1.041983
8704,BM,1951-11-30,0.388612,1.046032
...,...,...,...,...
22369,Mom12m,2023-08-31,9.140462,56.025695
22370,Mom12m,2023-09-29,7.178104,60.047278
22371,Mom12m,2023-10-31,6.032221,63.669462
22372,Mom12m,2023-11-30,6.939888,68.088051


In [19]:
# Save the cleaned and processed DataFrame
port_osap_clean.to_csv('../data/processed/OAP_signal_returns.csv', index=False)
print("File saved successfully.")


File saved successfully.
